In [156]:
# import the necessary packages
#from pyimagesearch import imutils
import numpy as np
import argparse
import cv2
from PIL import Image
import glob
import matplotlib.pyplot as plt

# define the upper and lower boundaries of the HSV pixel
# intensities to be considered 'skin'
lower = np.array([0, 48, 80], dtype = "uint8") # light shade of blue
upper = np.array([20, 255, 255], dtype = "uint8") # dark shade of blue
white = np.array([255,255,255],dtype="uint8" )
black = np.array([0,0,0],dtype="uint8" )

# ---------------------------------------------------------------------------------------------------------------
### images at various angles added to the list
# ---------------------------------------------------------------------------------------------------------------

In [149]:
image=[]
for ifile in glob.glob("D:/CV/seqcolor/*jpg"):
    image.append(ifile)
print(len(image))
print(image)

55
['D:/CV/seqcolor\\person01140+0-90.jpg', 'D:/CV/seqcolor\\person01141+0-75.jpg', 'D:/CV/seqcolor\\person01142+0-60.jpg', 'D:/CV/seqcolor\\person01143+0-45.jpg', 'D:/CV/seqcolor\\person01144+0-30.jpg', 'D:/CV/seqcolor\\person01145+0-15.jpg', 'D:/CV/seqcolor\\person01146+0+0.jpg', 'D:/CV/seqcolor\\person01147+0+15.jpg', 'D:/CV/seqcolor\\person01148+0+30.jpg', 'D:/CV/seqcolor\\person01149+0+45.jpg', 'D:/CV/seqcolor\\person01150+0+60.jpg', 'D:/CV/seqcolor\\person01151+0+75.jpg', 'D:/CV/seqcolor\\person01152+0+90.jpg', 'D:/CV/seqcolor\\person02140+0-90.jpg', 'D:/CV/seqcolor\\person02141+0-75.jpg', 'D:/CV/seqcolor\\person02142+0-60.jpg', 'D:/CV/seqcolor\\person02143+0-45.jpg', 'D:/CV/seqcolor\\person02144+0-30.jpg', 'D:/CV/seqcolor\\person02145+0-15.jpg', 'D:/CV/seqcolor\\person02146+0+0.jpg', 'D:/CV/seqcolor\\person02147+0+15.jpg', 'D:/CV/seqcolor\\person02148+0+30.jpg', 'D:/CV/seqcolor\\person02149+0+45.jpg', 'D:/CV/seqcolor\\person02150+0+60.jpg', 'D:/CV/seqcolor\\person02151+0+75.jpg'

### --------------------------------------------------------------------------------------------------------------------------------------------------------------
### generating mask for each of the images
### --------------------------------------------------------------------------------------------------------------------------------------------------------------

In [150]:
m=[]
def mask(imageM):
    for file in imageM:
        img= cv2.imread(file)
        converted= cv2.cvtColor(img,cv2.COLOR_BGR2HSV) #BGR2HSV hue saturation and value
        skin_mask= cv2.inRange(converted,lower,upper) #extract the part which is between the specified range colors
        kernel= cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11,11)) #structuring element of eliptical shape and 11,11 dimension
        skin_mask= cv2.dilate(skin_mask,kernel,iterations=2) #dialation of the edges
        skin_mask= cv2.GaussianBlur(skin_mask, (3,3),0) # smoothening of the edges
        skin_mask=  cv2.cvtColor(skin_mask,cv2.COLOR_GRAY2RGB) #converting the image back to 3 channel image
        mat= np.array(skin_mask)
        rows,cols,channels= mat.shape
        for i in range(rows):
            for j in range(cols):
                if(mat[i,j,1]!=0):
                    mat[i,j,1]=210 #color shade nearest to skin color
            
        for i in range(rows):
            for j in range(cols):
                if(mat[i,j,2]!=0):
                    mat[i,j,2]= 210
        mat= cv2.cvtColor(mat,cv2.COLOR_BGR2RGB) #converting BGR image back to RGB
        m.append(mat)
    #print(m)
    return m

### --------------------------------------------------------------------------------------------------------------------------------------------------------------
### generating the black and white outline of the face.
### --------------------------------------------------------------------------------------------------------------------------------------------------------------

In [151]:
bound=[]
def threshold(imageB):
    for file in imageB:
        img= cv2.imread(file)
        ed= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #color image to black and white because thresholding cannot be done on color
        ed= cv2.adaptiveThreshold(ed,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,15,3)
        ed= cv2.fastNlMeansDenoising(ed,50,7,21)
        ed=cv2.medianBlur(ed,3)
        ed= cv2.cvtColor(ed,cv2.COLOR_GRAY2RGB)
        bound.append(ed)
    #print(bound)
    return bound

In [152]:
value1= mask(image)
value2= threshold(image)

### --------------------------------------------------------------------------------------------------------------------------------------------------------------
### combining the mask and the outline with opacity being alpha and beta 
### --------------------------------------------------------------------------------------------------------------------------------------------------------------

In [153]:
def combine(Mvalue,Bvalue):
    count=0
    for (i,j) in zip(Mvalue,Bvalue):
        I= cv2.addWeighted(j,0.75,i,0.25,0.0)
        cv2.imwrite('mask/image'+str(count)+'.jpg',I)
        count=count+1
    return 0

    

In [154]:
print(len(value1))
print(len(value2))
combine(value1,value2)

55
55


0